# Sampler Testing
This notebook use to test which sampler works best for the given dataset.

In [2]:
import pandas as pd

from stroke_prediction.config import PROCESSED_DATA_DIR

train_data = pd.read_parquet(PROCESSED_DATA_DIR / "train-stroke-data.parquet")
test_data = pd.read_parquet(PROCESSED_DATA_DIR / "test-stroke-data.parquet")

In [3]:
from sklearn.model_selection import train_test_split

X_train_val = train_data.drop(columns=["stroke"])
y_train_val = train_data["stroke"]

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, random_state=42, stratify=y_train_val
)

X_test = test_data.drop(columns=["stroke"])
y_test = test_data["stroke"]

We will track the experiments using MLflow.

In [4]:
import mlflow

from stroke_prediction.util import get_or_create_experiment

mlflow.set_tracking_uri("https://mlflow.spikehub.win/")
experiment_id = get_or_create_experiment("Sampler Testing")

Before running the experiment, we will prepare the objective function that will be use to try each sampler.

In [5]:
from typing import Callable

import optuna
from imblearn.base import BaseSampler
from imblearn.over_sampling import (
    ADASYN,
    SMOTE,
    SMOTEN,
    SVMSMOTE,
    BorderlineSMOTE,
    RandomOverSampler,
)
from imblearn.under_sampling import (
    AllKNN,
    CondensedNearestNeighbour,
    EditedNearestNeighbours,
    NearMiss,
    NeighbourhoodCleaningRule,
    OneSidedSelection,
    RandomUnderSampler,
    RepeatedEditedNearestNeighbours,
    TomekLinks,
)
from sklearn.base import BaseEstimator
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier


class DummySampler(BaseSampler):
    def __init__(self):
        super().__init__()

    def _fit_resample(self, X, y):
        return X, y

    def fit_resample(self, X, y):
        return self._fit_resample(X, y)


oversampler = {
    "ADASYN": ADASYN,
    "SMOTE": SMOTE,
    "SMOTEN": SMOTEN,
    "SVMSMOTE": SVMSMOTE,
    "BorderlineSMOTE": BorderlineSMOTE,
    "RandomOverSampler": RandomOverSampler,
    "None": DummySampler,
}

undersampler = {
    "AllKNN": AllKNN,
    "CondensedNearestNeighbour": CondensedNearestNeighbour,
    "EditedNearestNeighbours": EditedNearestNeighbours,
    "NearMiss": NearMiss,
    "NeighbourhoodCleaningRule": NeighbourhoodCleaningRule,
    "OneSidedSelection": OneSidedSelection,
    "RandomUnderSampler": RandomUnderSampler,
    "RepeatedEditedNearestNeighbours": RepeatedEditedNearestNeighbours,
    "TomekLinks": TomekLinks,
    "None": DummySampler,
}

oversampler_list = list(oversampler.keys())
undersampler_list = list(undersampler.keys())

search_space = {"oversampling_method": oversampler_list, "undersampling_method": undersampler_list}

In [5]:
def create_objective(create_model: Callable[[], BaseEstimator], experiment_id: str) -> callable:
    def objective(trial: optuna.Trial) -> float:
        with mlflow.start_run(experiment_id=experiment_id, nested=True):
            oversampling_method = trial.suggest_categorical("oversampling_method", oversampler_list)
            undersammpling_method = trial.suggest_categorical(
                "undersampling_method",
                undersampler_list,
            )

            # Check if it has random_state parameter
            try:
                oversampler_instance = oversampler[oversampling_method](random_state=42)
            except TypeError:
                oversampler_instance = oversampler[oversampling_method]()

            try:
                undersampler_instance = undersampler[undersammpling_method](random_state=42)
            except TypeError:
                undersampler_instance = undersampler[undersammpling_method]()

            X_train_resampled, y_train_resampled = oversampler_instance.fit_resample(
                X_train, y_train
            )
            X_train_resampled, y_train_resampled = undersampler_instance.fit_resample(
                X_train_resampled, y_train_resampled
            )

            clf = create_model()

            clf.fit(X_train_resampled, y_train_resampled)
            y_pred = clf.predict(X_val)
            f1_score_val = f1_score(y_val, y_pred)

            mlflow.log_params(
                {
                    "oversampling_method": oversampling_method,
                    "undersampling_method": undersammpling_method,
                }
            )
            mlflow.log_metric("f1", f1_score_val)

        return f1_score_val

    return objective

## Decision Tree
First we will try a decision tree classifier to see how well it performs on the dataset for each sampler combination.

In [6]:
objective = create_objective(lambda: DecisionTreeClassifier(random_state=42), experiment_id)

run_name = "Decision Tree Run"

with mlflow.start_run(experiment_id=experiment_id, run_name=run_name, nested=True):
    study = optuna.create_study(
        direction="maximize", sampler=optuna.samplers.GridSampler(search_space)
    )
    study.optimize(objective, n_trials=len(oversampler_list) * len(undersampler_list))

    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_f1_score", study.best_value)

    mlflow.set_tags(
        tags={
            "project": "stroke_prediction",
            "optimizer_engine": "optuna",
            "model_family": "DecisionTreeClassifier",
        }
    )

[I 2025-05-28 21:20:53,852] A new study created in memory with name: no-name-13f820fd-2570-416c-bd53-33933dadccba
[I 2025-05-28 21:20:57,907] Trial 0 finished with value: 0.13157894736842105 and parameters: {'oversampling_method': 'SMOTEN', 'undersampling_method': 'RandomUnderSampler'}. Best is trial 0 with value: 0.13157894736842105.


🏃 View run crawling-tern-825 at: https://mlflow.spikehub.win/#/experiments/6/runs/091a24b1e3a84b389f2a902229c2cf2a
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6


[I 2025-05-28 21:21:00,477] Trial 1 finished with value: 0.0 and parameters: {'oversampling_method': 'SMOTEN', 'undersampling_method': 'RepeatedEditedNearestNeighbours'}. Best is trial 0 with value: 0.13157894736842105.


🏃 View run sedate-calf-260 at: https://mlflow.spikehub.win/#/experiments/6/runs/ed439eb297c1473ebbcf5a2aa1303f6a
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6


[I 2025-05-28 21:21:01,194] Trial 2 finished with value: 0.1111111111111111 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'TomekLinks'}. Best is trial 0 with value: 0.13157894736842105.


...

🏃 View run capricious-carp-12 at: https://mlflow.spikehub.win/#/experiments/6/runs/331c8436f6bd49d6b7d7ec0a4954f5cb
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6


[I 2025-05-28 21:27:36,585] Trial 68 finished with value: 0.08955223880597014 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'RepeatedEditedNearestNeighbours'}. Best is trial 29 with value: 0.22535211267605634.


🏃 View run unleashed-mare-671 at: https://mlflow.spikehub.win/#/experiments/6/runs/af0a18d760c249db829e34070e6cbc67
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6


[I 2025-05-28 21:27:37,610] Trial 69 finished with value: 0.13513513513513514 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'NeighbourhoodCleaningRule'}. Best is trial 29 with value: 0.22535211267605634.


🏃 View run nosy-elk-120 at: https://mlflow.spikehub.win/#/experiments/6/runs/033bf7b0cd564632b634c70989228b71
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6
🏃 View run Decision Tree Run at: https://mlflow.spikehub.win/#/experiments/6/runs/19649075a47f45e6955ff510c126c1a1
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6


## Random Forest
Next, we will try a random forest classifier to see if it can improve the performance over the decision tree and see if it compatitble with different sampler.

In [7]:
from sklearn.ensemble import RandomForestClassifier

search_space = {"oversampling_method": oversampler_list, "undersampling_method": undersampler_list}

objective = create_objective(lambda: RandomForestClassifier(random_state=42), experiment_id)

run_name = "Random Forest Run"

with mlflow.start_run(experiment_id=experiment_id, run_name=run_name, nested=True):
    study = optuna.create_study(
        direction="maximize", sampler=optuna.samplers.GridSampler(search_space)
    )
    study.optimize(objective, n_trials=len(oversampler_list) * len(undersampler_list))

    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_f1_score", study.best_value)

    mlflow.set_tags(
        tags={
            "project": "stroke_prediction",
            "optimizer_engine": "optuna",
            "model_family": "RandomForestClassifier",
        }
    )

[I 2025-05-28 21:27:38,201] A new study created in memory with name: no-name-a7492472-cb8e-4d9a-ac26-a369769acb23
[I 2025-05-28 21:27:40,835] Trial 0 finished with value: 0.0 and parameters: {'oversampling_method': 'SMOTEN', 'undersampling_method': 'RandomUnderSampler'}. Best is trial 0 with value: 0.0.


🏃 View run sassy-gnat-960 at: https://mlflow.spikehub.win/#/experiments/6/runs/a2edea8343fb4f65989a224d12d2bd61
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6


[I 2025-05-28 21:27:43,475] Trial 1 finished with value: 0.0 and parameters: {'oversampling_method': 'SMOTEN', 'undersampling_method': 'RepeatedEditedNearestNeighbours'}. Best is trial 0 with value: 0.0.


🏃 View run victorious-cow-21 at: https://mlflow.spikehub.win/#/experiments/6/runs/0319d41eebb24edeb8dd96c05f91fba3
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6


[I 2025-05-28 21:27:44,645] Trial 2 finished with value: 0.0 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'TomekLinks'}. Best is trial 0 with value: 0.0.


...

🏃 View run chill-auk-499 at: https://mlflow.spikehub.win/#/experiments/6/runs/6890796714f3460b8d45b4bd470e5103
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6


[I 2025-05-28 21:34:40,484] Trial 68 finished with value: 0.0 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'RepeatedEditedNearestNeighbours'}. Best is trial 8 with value: 0.2545454545454545.


🏃 View run rumbling-chimp-19 at: https://mlflow.spikehub.win/#/experiments/6/runs/61f153bc496f40ada5c57b523bb3a0b0
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6


[I 2025-05-28 21:34:41,915] Trial 69 finished with value: 0.0 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'NeighbourhoodCleaningRule'}. Best is trial 8 with value: 0.2545454545454545.


🏃 View run rumbling-stoat-252 at: https://mlflow.spikehub.win/#/experiments/6/runs/8cfb28d9d8804f78a19288344680f9e6
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6
🏃 View run Random Forest Run at: https://mlflow.spikehub.win/#/experiments/6/runs/80d240c6370246f993c0dd8e297cabee
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6


## K-Nearest Neighbors
Now, we will try a different model, K-Nearest Neighbors, to see if it can perform differently with the samplers.

In [8]:
from sklearn.neighbors import KNeighborsClassifier

objective = create_objective(lambda: KNeighborsClassifier(), experiment_id)

run_name = "Neighbors Classifier Run"

with mlflow.start_run(experiment_id=experiment_id, run_name=run_name, nested=True):
    study = optuna.create_study(
        direction="maximize", sampler=optuna.samplers.GridSampler(search_space)
    )
    study.optimize(objective, n_trials=len(oversampler_list) * len(undersampler_list))

    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_f1_score", study.best_value)

    mlflow.set_tags(
        tags={
            "project": "stroke_prediction",
            "optimizer_engine": "optuna",
            "model_family": "KNeighborsClassifier",
        }
    )

[I 2025-05-28 21:34:42,478] A new study created in memory with name: no-name-8de6558e-d36e-4324-b6c5-faf47efe444c
[I 2025-05-28 21:34:45,077] Trial 0 finished with value: 0.047058823529411764 and parameters: {'oversampling_method': 'SMOTEN', 'undersampling_method': 'RandomUnderSampler'}. Best is trial 0 with value: 0.047058823529411764.


🏃 View run judicious-mule-692 at: https://mlflow.spikehub.win/#/experiments/6/runs/da919f6fe1e24a51ac72099a958fa463
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6


[I 2025-05-28 21:34:47,411] Trial 1 finished with value: 0.04938271604938271 and parameters: {'oversampling_method': 'SMOTEN', 'undersampling_method': 'RepeatedEditedNearestNeighbours'}. Best is trial 1 with value: 0.04938271604938271.


🏃 View run classy-bass-815 at: https://mlflow.spikehub.win/#/experiments/6/runs/621679795f76450ea35b24da86c34dd6
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6


[I 2025-05-28 21:34:48,050] Trial 2 finished with value: 0.16176470588235295 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'TomekLinks'}. Best is trial 2 with value: 0.16176470588235295.


...

🏃 View run unequaled-doe-850 at: https://mlflow.spikehub.win/#/experiments/6/runs/bbc491a7466946edbf8f1f87576fea32
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6


[I 2025-05-28 21:41:27,875] Trial 68 finished with value: 0.17054263565891473 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'RepeatedEditedNearestNeighbours'}. Best is trial 46 with value: 0.19330855018587362.


🏃 View run youthful-gnat-289 at: https://mlflow.spikehub.win/#/experiments/6/runs/295579b6e5bb4b57b06f4eeb7debbe4a
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6


[I 2025-05-28 21:41:28,776] Trial 69 finished with value: 0.17543859649122806 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'NeighbourhoodCleaningRule'}. Best is trial 46 with value: 0.19330855018587362.


🏃 View run placid-grouse-887 at: https://mlflow.spikehub.win/#/experiments/6/runs/565fab1013a0407b9c9beff338ada4d9
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6
🏃 View run Neighbors Classifier Run at: https://mlflow.spikehub.win/#/experiments/6/runs/6eb8d48a100442ebb8ec019b63984518
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6


## CatBoost
Finally, we will try CatBoost classifier which we likely to use in production. We will see how it performs with the samplers and if it can outperform the previous models.

In [6]:
from catboost import CatBoostClassifier, Pool

val_data = Pool(
    data=X_val,
    label=y_val,
)

test_data = Pool(
    data=X_test,
    label=y_test,
)


def catboost_objective(trial: optuna.Trial) -> float:
    with mlflow.start_run(nested=True, experiment_id=experiment_id):
        oversampling_method = trial.suggest_categorical("oversampling_method", oversampler_list)
        undersammpling_method = trial.suggest_categorical(
            "undersampling_method",
            undersampler_list,
        )

        # Check if it has random_state parameter
        try:
            oversampler_instance = oversampler[oversampling_method](random_state=42)
        except TypeError:
            oversampler_instance = oversampler[oversampling_method]()

        try:
            undersampler_instance = undersampler[undersammpling_method](random_state=42)
        except TypeError:
            undersampler_instance = undersampler[undersammpling_method]()

        X_train_resampled, y_train_resampled = oversampler_instance.fit_resample(X_train, y_train)
        X_train_resampled, y_train_resampled = undersampler_instance.fit_resample(
            X_train_resampled, y_train_resampled
        )

        clf = CatBoostClassifier(
            random_seed=42,
            verbose=0,
            auto_class_weights="Balanced",
            eval_metric="F1",
        )

        clf.fit(X_train_resampled, y_train_resampled, eval_set=val_data, early_stopping_rounds=100)
        y_pred = clf.predict(X_val)
        f1_score_val = f1_score(y_val, y_pred)

        mlflow.log_params(
            {
                "oversampling_method": oversampling_method,
                "undersampling_method": undersammpling_method,
            }
        )
        mlflow.log_metric("f1", f1_score_val)

    return f1_score_val

In [7]:
run_name = "Catboost Classifier Run"

with mlflow.start_run(experiment_id=experiment_id, run_name=run_name, nested=True):
    study = optuna.create_study(
        direction="maximize", sampler=optuna.samplers.GridSampler(search_space)
    )
    study.optimize(catboost_objective, n_trials=len(oversampler_list) * len(undersampler_list))

    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_f1_score", study.best_value)

    mlflow.set_tags(
        tags={
            "project": "stroke_prediction",
            "optimizer_engine": "optuna",
            "model_family": "CatBoostClassifier",
        }
    )

[I 2025-05-28 22:12:08,049] A new study created in memory with name: no-name-9e1624f2-241f-4eb6-9de0-015f038f7031
[I 2025-05-28 22:12:11,316] Trial 0 finished with value: 0.16304347826086957 and parameters: {'oversampling_method': 'SMOTEN', 'undersampling_method': 'RandomUnderSampler'}. Best is trial 0 with value: 0.16304347826086957.


🏃 View run caring-ape-983 at: https://mlflow.spikehub.win/#/experiments/6/runs/c646ccf03c764fd1ba71c528b8d61018
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6


[I 2025-05-28 22:12:14,761] Trial 1 finished with value: 0.13008130081300814 and parameters: {'oversampling_method': 'SMOTEN', 'undersampling_method': 'RepeatedEditedNearestNeighbours'}. Best is trial 0 with value: 0.16304347826086957.


🏃 View run big-dog-334 at: https://mlflow.spikehub.win/#/experiments/6/runs/fd9b1f4602c44673990fde2f0a113d2d
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6


[I 2025-05-28 22:12:16,061] Trial 2 finished with value: 0.2898550724637681 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'TomekLinks'}. Best is trial 2 with value: 0.2898550724637681.


...

🏃 View run dashing-yak-913 at: https://mlflow.spikehub.win/#/experiments/6/runs/89db926862304a69a5f81c21314b2517
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6


[I 2025-05-28 22:19:44,539] Trial 68 finished with value: 0.3125 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'RepeatedEditedNearestNeighbours'}. Best is trial 13 with value: 0.3125.


🏃 View run peaceful-worm-504 at: https://mlflow.spikehub.win/#/experiments/6/runs/0d5feae5ca73407ba1b1e53bc2f0af26
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6


[I 2025-05-28 22:19:46,359] Trial 69 finished with value: 0.30985915492957744 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'NeighbourhoodCleaningRule'}. Best is trial 13 with value: 0.3125.


🏃 View run enchanting-hare-78 at: https://mlflow.spikehub.win/#/experiments/6/runs/fa5bb8484bd64ff198a04b8ad9d6cd69
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6
🏃 View run Catboost Classifier Run at: https://mlflow.spikehub.win/#/experiments/6/runs/2f3dc0afd5e54fd0956f7ff0b470b389
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/6


After testing the sampler we find the for the oversampling methods, the best one is the BorderlineSMOTE, and for the undersampling methods, the best one is the NeighbourhoodCleaningRule.

In [ ]:
study.best_params, study.best_value

({'oversampling_method': 'BorderlineSMOTE',
  'undersampling_method': 'NeighbourhoodCleaningRule'},
 0.3076923076923077)